# **Azure 语音转文本例子-字幕生成**

**字幕文件有srt/vtt格式，你可以通过Speech SDK 读取连续的语音文件**

In [38]:
!pip3 install srt

In [39]:
import azure.cognitiveservices.speech as speechsdk
import os
import time
import pprint
import json
import srt
import datetime

In [40]:
speech_key, service_region = "", ""
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)


In [41]:
audio_input = speechsdk.AudioConfig(filename="demo01.wav")


# RATE = 48000
# KHz_RATE = int(RATE/1000)
# CHUNK = int(RATE)

# audioFormat = speechsdk.audio.AudioStreamFormat(
#         samples_per_second=KHz_RATE, bits_per_sample=16, channels=2)

# stream = speechsdk.audio.PushAudioInputStream(audioFormat)

#audio_input = speechsdk.AudioConfig(stream=stream)


In [42]:
speech_config.speech_recognition_language="zh-CN"
speech_config.request_word_level_timestamps()

speech_config.enable_dictation()
speech_config.output_format = speechsdk.OutputFormat(1)

speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)

In [43]:
all_results = []
results = []
transcript = []
words = []

In [44]:
def handle_final_result(evt):
    import json
    all_results.append(evt.result.text) 
    results = json.loads(evt.result.json)
    transcript.append(results['DisplayText'])
    confidence_list_temp = [item.get('Confidence') for item in results['NBest']]
    max_confidence_index = confidence_list_temp.index(max(confidence_list_temp))
    words.extend(results['NBest'][max_confidence_index]['Words'])

In [45]:
done = False

def stop_cb(evt):
    print('CLOSING on {}'.format(evt))
    speech_recognizer.stop_continuous_recognition()
    global done
    done= True

In [46]:
speech_recognizer.recognized.connect(handle_final_result) 
#Connect callbacks to the events fired by the speech recognizer    
speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
# stop continuous recognition on either session stopped or canceled events
speech_recognizer.session_stopped.connect(stop_cb)
speech_recognizer.canceled.connect(stop_cb)

In [47]:
speech_recognizer.start_continuous_recognition()

while not done:
    time.sleep(.5)

SESSION STARTED: SessionEventArgs(session_id=36543eb3f5fa4e70a1348edc54fbb52c)
RECOGNIZING: SpeechRecognitionEventArgs(session_id=36543eb3f5fa4e70a1348edc54fbb52c, result=SpeechRecognitionResult(result_id=7805a08ba9ba4934ad2925c88d53c6f2, text="同学们好", reason=ResultReason.RecognizingSpeech))
RECOGNIZING: SpeechRecognitionEventArgs(session_id=36543eb3f5fa4e70a1348edc54fbb52c, result=SpeechRecognitionResult(result_id=3ce89dc049ad45a9844c49a9ba517db0, text="同学们好我们", reason=ResultReason.RecognizingSpeech))
RECOGNIZING: SpeechRecognitionEventArgs(session_id=36543eb3f5fa4e70a1348edc54fbb52c, result=SpeechRecognitionResult(result_id=66c0a92fc72a485aa6711d002a6938ee, text="同学们好我们下面", reason=ResultReason.RecognizingSpeech))
RECOGNIZING: SpeechRecognitionEventArgs(session_id=36543eb3f5fa4e70a1348edc54fbb52c, result=SpeechRecognitionResult(result_id=93cc3d8ce8fe489ab12710cdb8b0b1e1, text="同学们好我们下面开始", reason=ResultReason.RecognizingSpeech))
RECOGNIZING: SpeechRecognitionEventArgs(session_id=36543e

KeyboardInterrupt: 

RECOGNIZING: SpeechRecognitionEventArgs(session_id=36543eb3f5fa4e70a1348edc54fbb52c, result=SpeechRecognitionResult(result_id=a574a7852bf24fa39744b8121bdc6a8d, text="同学们好我们下面开始上课了这一个多月来我们都在共同的探讨一个课题就是小学数学信息化教学设计同学们经过了5次的设计迭代都形成了自己的教学设计的作品", reason=ResultReason.RecognizingSpeech))
RECOGNIZING: SpeechRecognitionEventArgs(session_id=36543eb3f5fa4e70a1348edc54fbb52c, result=SpeechRecognitionResult(result_id=f4cfa679680340f591177c54eb85b106, text="同学们好我们下面开始上课了这一个多月来我们都在共同的探讨一个课题就是小学数学信息化教学设计同学们经过了5次的设计迭代都形成了自己的教学设计的作品特别是", reason=ResultReason.RecognizingSpeech))
RECOGNIZING: SpeechRecognitionEventArgs(session_id=36543eb3f5fa4e70a1348edc54fbb52c, result=SpeechRecognitionResult(result_id=a6a45497bb3b45239000a30f05b0f0b4, text="同学们好我们下面开始上课了这一个多月来我们都在共同的探讨一个课题就是小学数学信息化教学设计同学们经过了5次的设计迭代都形成了自己的教学设计的作品特别是十二五", reason=ResultReason.RecognizingSpeech))
RECOGNIZING: SpeechRecognitionEventArgs(session_id=36543eb3f5fa4e70a1348edc54fbb52c, result=SpeechRecognitionResult(result_id=3052d47fc8464c1a99ba21

In [ ]:
print("Printing all results:")
print(all_results)

speech_to_text_response = words

In [ ]:
def convertduration(t):
    x= t/10000
    return int((x / 1000)), (x % 1000)

In [ ]:
bin = 3
duration = 0 
transcriptions = []
transcript = ""
index,prev=0,0
wordstartsec,wordstartmicrosec=0,0
for i in range(len(speech_to_text_response)):
    transcript = transcript  + speech_to_text_response[i]["Word"]
    print(transcript)
    if(int((duration / 10000000)) < bin): 
        wordstartsec,wordstartmicrosec=convertduration(speech_to_text_response[i]["Offset"])
        duration= duration+speech_to_text_response[i]["Offset"]-prev
        prev=speech_to_text_response[i]["Offset"]
    else : 
        index=index+1
        transcriptions.append(srt.Subtitle(index, datetime.timedelta(0, wordstartsec, wordstartmicrosec), datetime.timedelta(0, wordstartsec+bin, 0), transcript))
        duration = 0 
        transcript=""

In [ ]:
transcriptions.append(srt.Subtitle(index, datetime.timedelta(0, wordstartsec, wordstartmicrosec), datetime.timedelta(0, wordstartsec+bin, 0), transcript))
subtitles = srt.compose(transcriptions)
print(subtitles)
with open("./ql_subtitle.srt", "w", encoding="utf-8") as f:
    f.write(subtitles)

In [ ]:
strContent = ''
for speech in all_results:
    strContent += speech 

print(strContent)

In [ ]:
import re

In [ ]:
strContent='同学们好，我们下面开始上课了。这一个多月来，我们都在共同的探讨一个课题，就是小学数学信息化教学设计，同学们经过了5次的设计迭代，都形成了自己的教学设计的作品，特别是12位同学在智慧教室的环境之下，将自己的作品加以实施，然后开展了信息化教学，同学们都做得非常不错，那我们一起来回忆一下大家在做教学设计的时候一般是按照什么进程来开展的。先做前端分析，然后做过程，设计好其实对前端分析我们说的还是蛮详细的，但是过程设计是让同学们在实践当中，然后自己去体悟的，然后这节课我们加以总结和提炼好，那你们回想一下做过程设计的时候，呃，一般又是按照什么进程来操作的？过程设计就前端分析，我们已经做完了，下面我要安排我的教学过程了啊。那智飞你说一下。你是怎么做的？我一般就是先根据。一下我的教学目标，嗯？根据我的目标跟主题，然后进行下一步的内容。嗯，我的整个，呃，教学策略的选择最后再进行一个教学。呃，一个评价啊，评价，其实你刚才说到的目标重难点啊，包括评价测评项目的设计，我都可以在前端分析来做，但现在你要安排你的教学流程啊，其实就是你的过程的设计啊。那你刚才说你会安排你的纵向上面的教学流程啊，那么横向上面呢，就每一个环节的。我会。呃，今天的话就是安排首先就是它根据内容上要写连贯起来啊，要有教学内容，然后每一个环节的内容要连贯，然后还有，然后通过同学的分组讨论或者怎么样，就是或者那个翻转课堂的形式，让他们不同的方式进行不同的选择，通过他们自己的方法来选择他们合适的学习内容，然后再根据教师的点拨来进行。好的，好，我明白了行，谢谢。谢志飞，好也就是说，中向上面，我要想我的教学的流程以及时间的分配，然后横向上每一个环节的教学内容，包括刚才自费提到的学生的学习方法啊，老师的教学方法，以及，呃，你要采用的技术手段，当然还要考虑我前端分析做的这个目标，我要在哪一个环节达成，要用什么样的测评项目帮助我去达成考察学生的这一个目标的，呃，实现情况还有重难点，我要在哪一个环节加以实现和突破？啊，所以同学们要有这种系统化的设计思维的啊。那么对于这个教学过程，你们这样统筹安排过之后，你有没有思考一个问题，比如说这个教学环节，我这节课安排了5个教学环节，那我为什么要安排这5个教学环节？后来。层层突破，教学重难点。达到自己的教学目标，那你安排了5个环节，我为什么要一定要安排这5个环节就一定能够达到我的教学目标呢？建平，建平你说一下就是你安排这个教学环节。假如我这节课安排5个环节，你为什么这样安排呢？其实我觉得就是根据我们的前端分析啊，比如说学习分析学生已有的知识点，然后去设计一个合适的情境，或者说教学引入导入，嗯，然后再根据我们导入的内容去寻找一个点来。其实我们的教学内容。这样子一步步实现我们的教学目标哦。没问题就是。那我为什么第一环节一定要安排这个情境导入的？嗯，就是。我认为就是在我刚开始做教学设计的时候，就有参考别人的一些教学设计，我觉的可以借鉴，他们就是比较合理的过程安排。嗯，然后你你觉得是能够实现你的目标的，也能突破教学重难点是吗？对好的，好请坐，谢谢。建平也就说我刚开始作为新手老师的时候我可能会借鉴一些优秀的教学设计，包括我们在平台上面绘图，呃，推送名师的课例啊，教学设计给同学们做参考，这没有问题的，这是我们一条选择的路径。那我们再参考别人的教学设计的过程当中，我们发现我们能够突破难点，能够实现我的教学目标，我也可以把这节课完整的实施出来的。那好，这是一条路径，还有没有其他的路径。其实我们在讲教学设计之前我们讲过教材的分析，我们说教材是按照一定的逻辑体系来编排的，那我们可以按照教材的某一个课时的，它的安排的逻辑顺序对应着安排我的教学的顺序，这是不是也是可以作为第2条来设计的路径的？那当然以后我们成为能手，教学能手熟手老师的时候我们就可以用教材来创造性地去安排我的教学过程，不是简简单单做一个复印机的啊，但至少对于新手老师来说，我可以来教教材。是我第2条的选择路径，那今天呢，我们就一起来探寻我们的第3条路径。好。呃，我们说每个人做教学设计的入手点是可以不一样的。如果我们按照这个进程来操作的话，其中有一部分我们没有说就是关于教学模式的设计。什么是教学模式？其实你们一点都不陌生，之前我翻阅了你们以前学过的一些课程，包括现代课程与教学论，还有当代教育心理学，其实里面都有关于教学模式的这一个版块的内容，那和康老师给出了一个定义，在学界还是比较被广泛使用的。他说，教学模式是在一定的教育思想教学论学习论的指导之下，在某种环境当中展开的教学活动进程的稳定结构形式。那这地方的教学活动进程就被简称为是教学过程。所以说，从某种角度上来看，教学模式的设计就是帮助我们来安排。唉，教学过程，或者换句话说，你们如果能够把以前课程当中修得的一些教学模式合理的用在我们小学数学教学设计过程当中，就可以使得过程设计呢？更加。更加完善，更加有理有据。因为教学模式就是嫁接理论与实践的一个桥梁的，所以我们可以去学以致用。那这节课呢，我们在一起共同的来探讨这一个话题。好，那么为了增强课程的一个实践环节，我们把一部分的朴实性的知识呢放在课前让同学们去自学了。自学的效果如何？通过测试题呢？老师检测了一下发现有2个问题，失分率很高，一个就是关于数学教学模式的来源，还有一个就是针对这不同的来源，你们举出相应的案例，那下面我们就对课前的相关内容呢？我们再做一个巩固和深化好，首先是关于数学教学模式的概念，本质的啊，我们说我们在做教学设计的时候。呃，指导小学生来学某一个数学概念，我们绝对不会让他把定义背下来就可以了，我们要让它通过定义呢。把握住这个概念的本质属性，然后会去理解和应用。同样我们自己来学一个概念的时候，你也要把握它的最本质的含义。那数学教学模式呢？是包含3层含义的。第一层，他强烈的依附于某种理论和思想，包括数学课程，标准的理念的不同的理论思想带来的教学模式的形态是不一样的。那么第二呢，它受环境和资源的约束。比如说你们在智慧课堂之下开展，呃，教学实施，你会发现。智慧课堂环境之下的师生，就人与人之间和人与技术之间的关系较传统的多媒体课堂之下，他也会发生转变，而这个人与人之间的关系，师生的关系又是教学模式的一个很重要的核心要素的，对不对啊？所以它会受到这些影响。当然最明显的一个特征就是它反映了一些关系和结构。所以你回想你们以前学到的教学模式，它一定是有一个流程链条的，这个流程链条其实就是代表了我教学的一个纵向上的进程的。那么横向上面呢？它反映的是教学系统各要素之间的一个组合关系，那这个纵向时间维度和横向的空间维度相互交错，就形成了教学活动进程的稳定结构，那是不是和我们的教学过程设计是相得益彰的？所以可以指导我们做过程设计。当然，教学模式并不是仅仅这样的一个教学程序链条这么简单的事情，因为一个完整和科学的教学模式啊，它是包含各个构成要素的，那所以我们拿到一个教学模式，比如说赫尔巴特的五段教学法。哦，我看到了他有5个程序那是不是这5个程序就可以套用在我的所有的课程教学当中呢？那你要理解这个教学模式，它本身的理论基础，他的反应的师生关系，以及他想达到的终极的教学目标等等这些要素的内涵，否则你就会变成张冠李戴，他不一定就适合在你的这个课程当中，因为你这个课时可能想达成一个目标和他的目标不一致的，甚至你自己秉持的理念也不一致，所以选择的时候呢，你要对教学模式有深入的理解。你的选择才会有针对性的好，那我们学习的时候也要追本溯源，刨根问底，对每一个知识要点呢，都要有深入的理解。那么第3个就是关于模式的来源，那经过你们测试之后，你们自我的纠错，那现在应该知道数学教学模式的来源是由。归纳而得，演绎推理得来的，那么通过理论演绎推来的教学模式，一定要经受得住。时间的检验，你像西方有很多教学模式，那引到国内之后，它可能存在水土不服的现象，那我必须要经过实践的检验才是否能够用在我们的数学教育领域。那同样一位老师，他有很非常丰富的教学经验，有自己的教学智慧，他想凝练出属于自己的教学模式，但这个教学模式他要能够符合教育教学规律，有持久的生命力，能够被其他人所借鉴，他就必须要寻找自己合理的教学的理论的基础和思想的基础的好。那么针对，呃，不同的来源。我们来举一些案例。那么对于演绎而得的呢？这个地方呢？就取出一个数学问题解决。教学模式，这个模式的理论基础是波利亚的解题理论，大家对波利亚很熟悉啊，匈牙利的伟大数学家，他出版了很多风靡全球的著作，其中有一本同学们一定要去读的就是怎样解题来指导小学生们来进行问题解决。那有研究者呢，就根据解题理论的这个4个环节推演，得到了针对数学问题解决这种特殊的课型可以采用的问题解决的教学模式。那么在纵向上面我们可以安排这样的5个环节，当然你可以创造性地发挥，根据自己的实际情况去增补一些环节。那么至于横向上，每个环节要安排什么样的教学活动任务呢？你比如说弄清问题这个环节，我要设计什么样的问题，串来支架来指导学生弄清问题，那这时候你就要回过头来看解题理论啊，要知道他背后的理论基础，他这一个环节应该做什么。所以我们学习每一个教学模式对它的各个构成要素都要有深入的挖掘。然后第2个举的案例呢，是邱学华老师提出来的。小学数学尝试教学模式从20世纪70年代邱老师提出尝试教学模式，到最后形成它的尝试教育理论，整整用了54 50呃，年半个世纪的时间。那最后，这种教学模式不仅在数学学科领域可以用，也推演到了其他的学科领域的好。这50年，他就在做这一件事情。同学可能会问邱老师为什么会提出这种教学模式？那我们说实践归纳而得的教学模式，他一定是在教学实践过程当中他。有瓶颈了，有问题了。他想改革，想变化，然后就提出自己的改革的设想，然后经过实践的归纳总结，形成了一套教学的流程好。那同样邱老师也是在他成为一名小学数学老师之后呢，他发现教育现象当中存在一个很大的问题，就是不管老师怎么教，学生的学习效果都不好。后来他发现了原因就是在于始终是老师讲学生听，然后学生处于一种被动的学习状态啊。后来他就想，我能不能反其道而行之，我就颠倒一下教学的秩序。就是学生先学老师后教，感觉就是现在。翻转课堂的鼻祖一样啊。先学后教的好。那么当然它在推广的过程当中也受到很多人的质疑，也遇到很多的困难，但是他没有放弃他不断的去寻找自己这种教学模式的理论的根基，使得它更加符合规律，然后更加可靠啊。后来她在中国传统文化体系当中找到了思想的根基，你比如说易经当中提到了老师不要强迫孩子去学习童蒙求我，孩子迷惑了，有问题他会反助于求老师好，然后包括孔子孟子，还有现代教育家陶行知和叶圣陶先生。他们都提出主张学生进行尝试学习的这种思想的，所以邱老师说我是站在前人的肩膀之上，把这种尝试的思想用在了我的小学数学教学过程当中啊。那他的这个教学模式的流程是一共有7个环节，同学们如果感兴趣可以课后呢？在自学一下？那这就是一个典型的通过实践归纳而得的教学模式，那么也是一个把数学教育扎根在中国传统文化体系当中，然后扎扎实实地站在坐在我们的。国土上面的一种教育的模式啊。就像邱老师说的一样，人的一生当中能够做好一件事情就不简单了。中国的基础教育应该是世界一流的，所以我们要有这种文化的自信。当然这种自信是来源于一代一代的教育工作者们他们不懈的努力。所以我们也应该有这样的责任担当。那未来，刘老师也很期待能够听到你们命名的一种教学模式。而且在小学数学教育界能够广泛的流传，我相信一定是有那么一天的好。那么这是，呃，因为时间关系，我们就举2个案例，课后我们再推送一些资源，特别是我们数学教育领域啊，归纳总结得到的，这样的教学模式好，他们都有二30年的这样的生命的历程的啊。所以还是呃，非常不错的一些成果啊，那这样我们就把课前一些学的基本的内容呢复习巩固了一下啊，我们说学无止境，要精益求精，我这地方再给大家推荐2分资源，第一份呢，就是美国的两位学者，乔伊斯和韦尔。他们住的一本书，叫做的教学模式是1972年那到我2018年买这本书的时候，已经是第八版了啊。同学们可以买来看一下，因为非常经典这本书。呃，被在学界广泛认为是最早的对教学模式进行系统化研究的两位学者的。那么到了1983年的时候我国的钟启泉老师把他们的成果引入到国内，然后我们国内就开始对教学模式进行广泛的研究和实践的，所以他有这样的一个发展历程，大家可以买来看一下。啊，另外一本的是张店祝贺送暖气老师编著的，其中有一个章节是关于数学教学模式的好，这是客户大家可以学习一下，那这是课前我们学的内容，呃，我们可以回想一下，就是上一个星期我们选派了张译同学呢，到滑道附小呢，去上了一节公开课，那我们在大学的课堂进行了远程听评课校外导师呢？温老师也给我们做了点评。汪老师呢，在某一个时刻，她说了这样的一句话，看同学们还记不记得？然后问老师。他说。如果是这节课像我们这样的有经验的老师来上的话，呃，我可能会这样的设计，首先也是创设情境，引起冲突。我也问老师说，如果像我这样的老教师来上这节植树问题的课的话呢？我会这样去设计。然后张毅同学呢，是另外一种设计的思路。呃，我当时就问同学了嘛，他们其实在做什么？哦。同课异构的是不是？那我这地方的问题就是同课异构到底在异构什么？好，同学们之间可以讨论一下。因为一同课异构，是未来你们教学工作过程当中的一种教育常态的好，你们必须要知道什么是同课异构。到底下一步什么，我只有清楚了。我未来在实践这个教学研究活动的时候我就有方向性了，你们到底在易购是吗？我这地方随机挑一位同学来发表一下你的观点。这么巧？就是张译。我觉得同科异构最主要是，呃，呃。一个课程的教学过程的不同。嗯，其他方面的话就还有一些教学风格的策略等方面，但最主要的还是教学过程教学过程的，是吧？好，请坐好，有没有其他同学还有其他的观点要补充的。就是你觉得同课异构在异构什么？张毅的观点就是，过程呈现出来的过程不一样。哦，有这样的一个模式形态不一样，是吗？好，那其实模式的指导，我们的过程设计，那归根结底还是过程呈现的不一样的，但每个老师对过程的设计，他有自己的考量，有各种路径，比如说可以用模式来支持我，我可以按照教材编排的顺序，我可以凭借我的教育智慧的。最终呈现出来的是这个过程不一致啊。那其实作为一个同行，我们来观看我们能够看到的，捕捉到的就是他对教学过程的一种个性化的设计，包括背后折射出来的他的教育风格理念。他自己的智慧是这样对不对好。那么对于教学过程的设计，不同的人有不同的观点，有的人认为呢，就是它是一种呃，带有很强的个人色彩的艺术行为，我不需要有任何教学模式来匡复我，我拿到一个知识点，我就能很快的就把它呈现出来了。好，然后另外一种观点认为呢教学过程的设计应该做到有理有据，而教学模式就是嫁接理论与实践的桥梁，它可以使得我的教学过程呢，更加牢靠啊。所以我需要有模式的支持的。当然这是观点的两端。所以我们说我们必须要摒弃一种企图，就是我们在大学修得的一些教学模式，可以机械地套用在任何的课型上面去，那这只是要不得的啊。第2个的话呢，就是我们也要打破一种束缚，就是作为老师始终是基于经验的，我们还是需要有一定的理论的高度的。那特别是对于师范生之前，老师利用在大学的学习时光，能够掌握一些理论知识，然后用在我们的教学设计过程当中，至少你未来在同课异构的时候，你就可以多一条选择的路径了。好，那前面的话呢，我们对数学教学模式是什么以及我们为什么来修这个教学模式呢？我们有了这样的一个初步的认识，那接下来我们肯定要探讨的问题就是。怎么样？去爱去设计了对不对？我们知道是什么，为什么我们接下来就是要看如何去设计这个教学模式，当然这地方的设计并不是让同学们去原创性的。从理论的推演或者实践的归纳出来，因为我们现在还没有这样的机会和土壤，这地方的世界设计呢，是站在教学设计的这个语境之下，我们来进行过程的设计了，那么我们从众多的教学模式当中，我们怎么样去选择和应用来支持我的过程的设计？所以这地方强调的是，如何选择和应用教学模式，这是我们这节课要探讨的共同的核心问题。那这个问题呢，我们分两步走，首先来看一下设计方法，然后我们把这个方法用在实践当中，去体验一下好，呃。那我们现在问一个问题，就是你们觉得在选择和应用教学模式的时候会受到哪些因素的影响？因为我必须要考虑这些因素，我选择的时候可能才会有更加好的针对性。那么这个因素。影响的因素会有哪一些呢？哦。有个同学学琴，教学内容环境那么如意，你具体来说一下。嗯，就是。具体来说，教学内容，它其实反映的就是我想要达到教学目标，然后，呃，它是主要是知识技能的习得呢，还是实际能力的获得？嗯，比如说面积公式的推导，它必须是一个动手操作的，如果是按照接受然后记忆，就通过单纯的模仿和记忆，就不太喜欢。嗯，那？呃，还有我的是要，呃，根据学情，就像认识考虑学生的认知发展水平。一年级的学生，他其实不太适合那种动手操作合作探究的方式，它更多的是需要教师的引导，所以这些都是我要考虑的因素。好的还有其他什么信息，环境也是好的，好好谢谢如意的。首先我的教学目标，我们的目标定位，如果这节课是纯粹的，我想习得一个知识点，那这时候的话，老师的讲授的意味会比较足一些了。但如果我想培养学生的某一种动手的能力探究的能力，那我这一节课安排就一定有发现探究的活动的。模式选择是不一样的，包括学琴，刚才如意也提到了低年级的孩子，他的本身自我监控能力就没有那么强，那我不适合用这种探究的方法，可能适得其反的啊。当然刚才同学们也提到了，有环境的影响，比如说你们在智慧课堂的环境之下去选择教学模式，其实你们已经修读过尹老师的那一门慕课，它单独有一章是讲小学智慧课堂的教学模式的，对不对，那我就可以有针对性的去选择，所以他会受到很多因素的影响的啊，那么因为时间关系，我们这地方就针对两种特殊的课型。来探讨一下模式的选择和应用的过程。好那首先我们来看数学概念的教学模式的设计啊，你们之前都做过很多的教学设计了，至少在我这一门课上，你们已经过5次的迭代，先做前端分析，然后过程设计，最后形成作品，那么完成了一份教学设计，那你们在做这个概念设计的时候啊，你们的过程是怎么样去安排的？过程是怎么样安排的？有没有同学哪位同学是恰好是做概念教学的？唱了你是不是做概念教学的啊？哦，我之前做的一门课是叫做负数的认识啊，他也是属于其中的一个数学概念。嗯，那我的教学设计的话，我是啊，先让学生。呈现给学生呈现出一个复数的概念，让学生在脑子里初步的形成一个什么，是负数，嗯，什么事。我认为什么是复数，复数是什么，嗯，然后紧接着我会让他们举生活中的例子生活中的例子来外线出来。他对分数的认识。嗯，紧接着我会对他们说，诶，同学们，你们之前除了学负数之外，还学了什么树？嗯，会引导他们啊，说出零和正数，嗯，然后让他们所认为的这个负数何以？经学过的零和正数做一个对比，嗯，找到它们的不同点共同点，嗯，然后再进一步深化他们对负数的认识。嗯，接着，我就会，呃，直接的告诉他们复述是什么。嗯嗯，分数是啊，带有负号的啊，比零小的数。嗯，就会告诉他们，然后再设计一些练习的环节，明白让学生来进行进一步的好的，我明白，也就说你给出了大量的例子，然后和已有的旧观念当中的例子进行一个辨析。发现它的特征，然后我下定义啊，就会有例子的巩固练习巩固你为什么这样去设计安排你的这样的教学环节啊。其实我这个教学环节的设计的话，其实跟前面见面同学提到的有类似的地方，就是因为我经验不太够吗？那我会更多的去选择参考别人的教案，然后集百家之城，然后把它合在一起的，看看教学设计当中你觉得你自己在实施的过程当中是否还是流畅的学生？这样。我觉的还是有一点小卡顿，因为我没有很去实操过她这些环节的来源。为什么要这样设计？嗯，我感觉我只是硬套的过来哦。把它重新地展现出来，并没有形成很牢固的那个理论的依据哦。好的，我明白你的意思了。没问题，今天我们学习过相关内容，过之后看能否部分的解答你的疑惑的好。好，那么刚才，呃。再问建平的时候我的问法是你作为老师你怎么样去设计这门课，你是怎么样去交的教他的过程的，但其实我们可以换一种角度来思考，就学生是怎么学的，儿童是怎么修的这个概念的，他学习概念是轻，是有什么样的形式的。其实你们以前在一些课程当中学过，就是儿童获得概念的形式。哦哦概念的同化和概念形成，所以我们换种角度，以学定教。孩子是这样学的，我们可否这样去交啊？那么有形成和同化这个概念是谁提出来的？这个概念的形成概念同化还记得是谁提出来吗？对同化顺应平衡是皮亚杰先提出来对不对，然后后来奥苏贝尔赋予它新的内涵，就提出了认知同化理论。还有概念形成的理论的了啊。那么同学们不错啊。都还记得好那什么是概念的形成，就是通过大量的例子，我们圭，呃抽象概括出同一类事物的本质属性的过程，我们就把它称为叫做概念的形成的啊，那么对应着我们就可以得到概念的教学，通过形成的路径。得到这样的环节的，其实从儿童的学习的心理推演到老师的教的这个流程的。这种推演的这个方法，或者说这种逻辑，其实你们以前学过一种教学模式，也是按照这个逻辑来的，不知道还记不记得。就是加涅，他根据认知加工理论对。加涅他根据认知加工理论，是把学生的学划分为9个环节，我们称为是。九大事件还记不记得，然后对应着老师的教就有9个环节，所以这种教学模式叫做对讲烈酒段教学模式的。那同样我们从儿童的概念形成的过程，我们可以推演概念教学的这一个过程。比如说第一个环节，我提供大量的例证，那就是要辨别这些刺激的模式，然后学生经过比较和类比去抽出共同属性。比如说我们待会举一个案例，就是，呃，认识平行四边形，认识平时变形有很多的例子，学生去找共同属性，他们会找到哪些共同的属性？2条边互相平行噢，边互相平行，还有呢。相等，对角相等。对边相等，然后他又是四边形，有4个角，有很多的共同的特点，是不是啊？但是我们会发现最后下定义的时候我们只是用了对本质属性的，那你可以回去想一下为什么不用脚来下定义，为什么要用边来下定义的？我们可以去思考一下这个问题，好，那么就抽象出本质属性。我初步形成概念，我接下来要进行概念的深化，我的教学还要多一些深化的，就是你要下定义，你要有概念的正反例的辨析好，然后再有概念的应用。所以我经历这样的教学的流程的啊，那么概念的同化的心理过程是怎么样的？就是我直接把。对我告诉你，孩子们，我们翻开课本吧，我们来读一读什么是平行四边形啊。那这一个定义当中，我解读一下它的最关键的属性是什么？我们圈1圈，画好咯。我就观念当中的一些知识，我们建立一下连接，然后最后我们来运用概念。所以童话就是用就关就呃你的知识结构当中的旧的观念，去接纳新概念的新观念的这样的一个过程的，就是概念的同化，所以你会发现形成和同化两种不同的教学路径。他们的。有区别啊，每一种这种模式，它蕴含的意味不同。比如说行程，我在教学过程当中会安排有什么样的活动？探索的发现的对不对，然后童话呢是一种。下定义，直接下定义给定义老师来，给定义。啊。所以他有老师的。讲授为主的哦。余老师讲的，作为一种接受式的意味在这地方，所以你会根据你自己的不同的设计来选择合适的方法。那下面呢，我们就用这个认识平行四边形。我们来举一下例子啊，举下例子，我们一起来看一下我们用不同的教学模式啊，我们来选择怎么样去运用在这一个课的教学的过程当中啊，那么这一节课的教学内容我们先做分析，简单的前端分析这个内容的话，它是一个概念课是吧？我们是一节概念课。那我们知道在我们的已有知识体系当中，呃，概念学生学习是有两种路径的，那我们的教学的路径也可以有2条选择的就至少我有2条选择，当然还有其他的教学模式。今天我们从这2条当中来思考一下，那你们看过颗粒了，课前都已经让你们预习过了，那么老师其实很明显，他用的是。形成的是不是用形成的方式来进行这个教学的？所以你的知识体系当中你要有这个知识，你才能够拿来去用它好。那我们当然还想再看一下课程标准，他是怎么说的。课程标准这一节课的要求是通过观察和操作认识平行四边形，所以我们用形成是不是可以的，也符合课程标准的理念的对不对啊？好，那这节课的教学目标是什么呢？你看课标他前面有经有一个过程性的要求呢，就是要通过哎对。通过这样的观察和操作，这样的活动让学生呢，去对认识我理解概念，同时呢，我也会去应用这个概念，是不是啊，这是我们的目标好，那同学可能会问了，老师，我用童话行不行？你看，在这个知识链条当中，平行四边形的特征，它处于什么位置，承上启下的位置对不对？前面我们已经在一年级，还有三年级都学过相关知识，然后后面我们还要五年级要去学这个面积公式的啊。那所以他已经有这种知识的前辈知识了，我用童话可不可以完全是可以的，你可以去尝试这样去设计它好，那么目标有了，我们看一下环境。哦，刚才如意也说了，我送目标送内容受环境，所以学琴的影响，这个地方的环境我们尝试着去考虑一下。在智慧课堂的环境之下啊，我们帮这位老师去想一想，有些环节我是否可以去优化它的教学。好在智慧教室的环境之下。好，那这些地方的话是我们一些前端的一些思考了。那下面我们对教学的程序。好，还有。活动。对应的每一个环节，我可能会采用什么样的方法，还有技术手段，我们都可以去考量一下，就是对整个教学的程序啊，我们来构建一下那首先程序。按照概念的形成这种路径的话，我们第一个环节可以做什么？提供例证的我举例子是不是啊，我举了大量的例子，这个例子在视频当中是谁来举的？对学生举的除了学生去老师可不可以取可以，那么我们举什么例子呢？哦，生活当中的例子还可以呢。可是。你看，我们前面已经在一年级对学过，我就可以用数学的表象了，我就用数学的例子来举可不可以呀？我可以去辨识哪些是平行四边形，所以可以老师即可以学生局，然后可以让学生呢？你来说一说，也可以老师提供呢？让学生来辨析一下，我来认一认都可以，但这个地方其实是可以实现一个呃，情感态度这个目标的，其实一个很好的契机的就是你无论老师举学生局如果去生活当中的实力，那其实。应该让学生能够看到数学与生活的联结的，比如说课本上面有一个例子特别好，就是那个停车位的。为什么用平行四边形？这地方不用？长方形的长方形不是更好倒车吗？为什么用平行四边形呢？我是用到数学的知识，我用在生活当中我设计的平行四边形了。你说。不然更容易出去，更容易出去一点是这样吗？有些地方他一定是做的，是那个长方形的，但有些地方他就用了平行四边形。你们有没有想过这个问题的会不会节省空间啊？吃的不管对，但是如果杜伟停车位，你这个面积有些在市中心的地方是不是啊？寸土寸金的地方，停车位没有那么多，那我如果斜着放的话，它其实可以节省恐惧空间的就行与行之间的空间的好，就是说有时候举例子的时候，也可以让学生感受到数学与生活的联结的啊。那当然这是题外话了。好举例子过之后，下一个环节形成的概念形成的第2个环节是什么呢？对我们要找属性了，所以这地方我要确定它的属性，那这地方我是经历了两步来走的哪两步的？先观察，观察之后，你要去猜一猜，是不是啊？诶，学生们，你要去猜一猜，想一想，观察一下他们有哪些共同的特点呢？啊。那他们会猜什么？就是那个角边角关系对边相等对边。平行，然后呢？是啊，对角相等好，那数学是讲究道理的，你猜玩过之后你要去验证，当然这地方不适用数学的那个演绎推理来证明了我这地方需要动手操作了啊。如果我猜是对边相等，我需要两粒两，如果对角相等，也是要量一量，那现在如果猜对边平行呢？呃，则易者对，有的可以的，但是如果给你一个模具很硬你没办法折。好，这个地方就出现问题了，你们以前学的时候是用三角板和直尺，这样意义的。但现在部编版教材改了，把这一部分那种淡化掉了，学生画平行四边形的时候，都有方格纸方格子他行与行之间距离相等，所以我只要保证边相等的时候，他就平行。对，就是数格子的了，他把这个话的淡化掉了，所以你们要看现在的部编版的教材和你们以前不一样，那他现在用什么样的方法？如果脱离了方格纸之后，怎么画，怎么去验证？哦，对了，就是在前一个课时的时候，它有一个活动，它有一个活动，其实隐含了一个特性，就是平行四边形之间的距离处处相等的。所以你就要去画那个垂线的。但，呃，现在我知道有些小学老师他们觉得这个很麻烦，所以在验证的时候仍然使用那个平移的办法的，所以以后你们根据你们，呃，在教学工作岗位上的具体情况，你们去处理，这个验证平行的这个方法了，好，那么订完属性过之后我要做什么事情了。对确认本质属性，然后同时去下定就是深化概念了。我就是需要下定义了，那下定义的时候。让学生先去对尝试，你说一说吧，什么是平行四边形对不对？好。然后我翻开课本，我来爱圈1圈读一读等等这些活动我都可以让学生来做一下。圈1圈，读一读好，那么下完定义之后，本身我还可以做一下正反力的一个辨析，但这地方老师没有做，因为他有他的时间的考量，但他同时做了一个活动，就是概念的运用了。是不是啊，给您用活动时让学生去画一画。在画的过程当中，有正例反例，其实也是可以辨析的好。所以下面一个呢，就是你要会应用的问题会应用的问题，就是你要去画一画，你画一个平行四边形。在画的过程中，你要不断的去回想他的定义的，会想他定义来验证我画的是否正确的好，那我们就基本上把这个环节按照形成的路径安排出来了，它的对应的教学活动的，那我们看一下老师会采用什么样的教学方法呢，就有一个很明显的，在这个环节它是让学生去。哎，合作的同伴合作的，是不是我一起去验证一下。所以有个合作的。当然我在这个画一画的时候有错力的纠正是让一个学生上来做一下演示的对不对？我让他来演示一下。当然还有其他的讲授式啊，讨论对话法都有很多的方法的，当然这2个会比较明显一些，那么好我们也可以考虑一下技术手段的优化，我这节课的教学，你们觉得哪个环节可以优化一下？可以优化我们学生的学。哦，动手操作这地方其实。嗯，就给动画做出来就很直观的学生来看是不是啊啊。特别是刚才那个平行的测这个检测是不是哎很好，还有没有？你们有没有留意到这个学生就做错的那个同学，他其实是坐在教室的偏后的位置的，而这个男生上来纠错的时候，他个子比较高，他把整一个纸呢是给她遮住的，你如果这个地方我可以怎么去操作，就让全班同学都能够看到这个纠错的过程的。对了，我用手机或者拍呢，我去投影投到大屏幕上，学生就能看清楚这个过程了，哎，所以我可以去考虑技术的优化，这个地方我可以做投屏啊，这个技术可以介入到你的优化。好，那么下面我们一起来看一下啊这位老师他是怎么样安排他的教学过程的。那因为同学们都看过视频，我这地方会过得比较快一些。李老师给大家带来了一位老朋友，想知道吗？因为在一年级学过，所以一看就知道了是什么事。好，那么呢，让同学们去取例子很多生活当中的例子啊。把这些例子呢，我们抽取出来，这就是我们这节课要学的平行四边形，所以第一个环节就像刚才同学们说的呀，我举例子，我引出我这个新课那是举的是生活当中的例子，是让学生来举的那下一个环节呢？好行，这就是他的形象代言人。平行四边形，ABCC。我们看一看。他有几条屏？第一。嗯，就让学生去看它有什么特性了。那这地方的特性呢，是集中在边上面的。其实老师他是有预设的，他有预设，他引导同学去说有几条边，谁和谁是对边，然后边有什么样的特性。所以学生去猜想。当然我们看这些优质的颗粒的时候我们自己也有批判性的眼光，如果我来设计，那我会去怎么去设计他，那学生既然让他猜想，就可以让他大胆的去猜想，除了右边的关系。还可以由角的关系了好。那最后老师说。是讲究道理的，那我们就需要去验证的通畅，所以下一个环节就是合作验证了。那我们之前看滴水实验那个视频的时候，他也是一个呃，很就是合作探究的这样的一个活动的，我们说你要有合作，你就必须要有这种支架，学生知道该怎么去操作，所以接下来就是合作来验证猜想了啊。验证猜想。好验证，玩过之后。放弃了平行四边形的对边，不仅平行，而且。那到底什么是PC4边形呢？嗯，那到底什么是平行四边形呢？好，所以我就形成了概念，我去尝试定义了啊。那尝试定义的过程当中就让学生呢？翻开课本去读一读，然后画一画，然后最后呢，我再来动手操作，然后去运用这一个概念的，所以经历了这样整一个的环节，我把这个概念进行了学习了啊。那么通过这样的一个课例，我们知道了概念教学呢是有两种模式的，至少我们知道有两种，一种是形成一种童话，那我们也知道用在教学过程当中的时候呢？我做完前面的前端分析，有一些影响因素的分析，然后我把环节用在我的。过程的当中好，然后再横向上面去安排我教学内容，我知道我怎么去设计，当然更主要的是我们要知道它背后的原理是什么。首先你自己要知道概念课的教学就是学生的，呃，心里的学习过程是有形成和同化，这是知识储备，你一定是要有的。然后推演到教学有两种方式，这个逻辑是通顺的，是可以这样推演的。然后第二呢，我要知道课程标准的一些基本理念来支持我这节课的设计的。当然第三你也要知道一些发展心理学的一些知识，其实你们之前学过这一门课程。对儿童的概念学习里面也有相关的阐述，不知道同学们还记不记得，其实儿童学习概念最后它会慢慢慢慢的过渡到以什么为主的。以童话为主的是以童话为主的最低年级的孩子。他开始起初一定是要有直接能够观察的事物，他才能去学习概念，然后慢慢过渡到我，凭借已有的知识经验，我凭借数学的表象，我不要具体的事例了，我也可以学概念了，再到后面的时候我就纯粹依赖于理性的材料，就是定义的课本当中材料，我就可以学概念了，所以低年级的时候你们需要提供一些直观的事物，直观的属性，帮助他们理解概念。但是到高年级的时候我可以从非直观的本质属性还有呢概念之间的上下位关系，种属关系就是童话的方式来学概念了，所以奥苏贝尔就是说对于学龄儿童来说获得概念的最本质的特征就是童话的可能会打破你们的一种认知，就是我们认为概念学习可能都是用形成是最好的，但是到高年级的时候他完全是可以用童话的方式去接受这种新的知识的，好可以把以前学过的一些相关知识呢，给他调动起来一起学习。那么这是概念课的教学。下面我们看一下原理课的教学原理。课教学也有两种路径，一种是用力知道原理，另外一种是用原理到例子，那前者就是我给你大量的例子，你去去辨析，然后去研究，发现了一些规律，然后后面那一个呢就是我把规律直接告诉你，适当的时候我证明一下，然后给你大量的例子，你去巩固和应用，那显然前者是发现式的学习，后者是接受式的学习，那你知道了这两种路径的基本定位之后。你就想我之前学过的教学模式有哪一些是带有发现的意味，哪一些带有接受的意味？那我可以去折取和应用了好，那在这地方呢，我们就呃，介绍两种大家非常熟悉的模式，对于接受你们就能联想到是。有意义的接受是奥苏贝尔的，然后呢？发现式的教学呃，对你们可以联想他们提出的教学模式，用在你的教学环节。其实你可以想象到你们平常做一些公式性质的教学的时候，其实也是按照这样的路径来的。我们提出问题让学生提出假设我们一起来验证，假设获得结论，最后我们把结论进行应用。其实你们都是按照这样操作的，但它背后的原理是什么？哦认知发现理论来支持的那么童话呢？呃，接受呢有意义的接受是奥苏贝尔提出来的奥数标题，所有提出有意义的接受学习之外，他还提出了一个很重要的教学策略。你们还记不记得？先行，组织者策略很好的，那么奥斯贝尔认为什么是有意义的学习？他说，发现式的学习也不一定全部是有意义的，而接受式的学习也不缺，全部都是七届的，所以所谓的有意义就是能够建立起新旧知识之间的联结，对关系和联结的。所以你会发现它的教学模式不是第一个环节，就让你感知教材，把定理呈现给你的，它会有一个前置的环节，就是发挥先行组织者的作用的好。你们可以回去呢，再去钻研一下。那下面我们来看一下这个颗粒是吴正宪老师的惩罚。分配律啊是一个公式也是一个原理规则的，大家都非常喜欢吴老师的课例，那，呃，课前你们已经看过视频，我们这地方呢就快速过一下。那么你看课标当中的要求是什么，探索并了解运算力，所以他有这个探索发现的过程。我们采用布鲁纳的发现式的学习方法是可行的，对不对啊？然后你看他的知识链条啊，他是在处于整数的这个知识链条当中，当然学了这个，呃，运算力之后对分数和小数的简便运算，他也是有帮助的。所以这个知识也是非常重要的。好，那我们看吴老师是怎么上的？他首先第一个环节是做什么？还记不记得？或者你联想1下发现式的教学模式，第一个环节我是要做什么？提出问题对不对，创设问题情境提出问题，那这地方吴老师，他就创设的问题情境，然后提问题是他提的吗？哦，学生来提的学生要有这种提出问题的能力好，你应该提出，但这节课呢，我就聚焦在2个问题上。这2个问题适合我，后面的乘法，分配律的结构是有关联的，那我们就解决这2个问题，所以提出问题了。那提出问题之后，学生来解决了，那么解决，所以下面就是让学生嘛去说一说你的算法。不同的学生有不同的算法，但是他最后的结果是一样的，而且代表的现实背景的意义也一样。比如说我都是在求这个花坛的面积，或者都是求花坛，一共有多少盆花啊，方法不一样，结果是一样，所以我可以大胆地在中间划上一个=的，那就出现了2条和乘法分配律公式一样结构的算是了，对不对？就出现了2条了。好，吴老师问，你们有没有感觉呢？好像举手的孩子没有那么多，那没有关系。我再让你经历一轮这样的问题解决他是不是提出一个问题，就是两面墙的问题求面积，那么后面又有一个孩子呢？又写出了这个算是我黑板上就有3个算式了，那你找到感觉了没有？那其实这种感觉就是什么？就是我们数学当中的让你表达出来的，规律的就是它那个规律的，所以提出问题，然后提出猜想。这个猜想就是那个规律的好。那么学生在探寻这个规律的时候，你会发现。好，我要2个学生上来写了2个学生，确实能写出来了。那吴老师就问，你怎么胆子这么大，敢在中间划=的呢？好，那个女生是怎么回答的？对我算的，我算出来了，两边确实相等，所以我可以大胆地划=的啊。那吴老师说了，那是这个，算是有很多的无穷无尽的。你今天写不完的呀，那永远写不完，那你永远也算不完。那我怎么样去可以解释这样的一个规律呢啊提出猜想，下面就要验证这个猜想，学生是用什么来验证的？对了乘法的意义的。你想A乘以C加上B乘以C，代表乘法的意义就是ADC相加。B1个C相加再求和，那一共有多少个CA加B1个C所以，我用乘法的意义，这个算理我能解释清楚你这一个猜想，那我这个规律就是可行的了，那么就验证了这个规律了啊，验证这个规律过之后，下面一个环节是什么？你想一下刚才的布鲁纳那个环节下一个环节就是要做。呃，你已经已经证明过了，已经算力证明过了。还没到应用还要做什么？下，第一下结论了，是不是啊？那你要把这个公式要抛出来了，对不对是吧？你要下结论，这个公式是什么样？这个规律，要用这个式子来表达了，学生就可以不管你数怎么样变化，你的背景怎么样变换，我这个规律都可以使用的了。是不是啊？那这个下结论是吴老师下的吗？是让学生是不是啊，你把你自己的这种感受你表达出来吧。有的孩子用纯粹的文字语言表达，对不对，那有的孩子呢？是用符号三角形五角星，还有的孩子，最后慢慢引导它是能够用字母来表达的，用字母来表达这一个规律的。所以，层层递进的我提出猜想验证猜想去表述这个猜想就下结论。最后一个环节就是要去应用，但这地方吴老师很显然他的公开课时间都很长，他没有这个应用的环节，他只是做了一个简单的迁移。和之前我们学过的乘法竖式当中的分配律做了这样的一个迁移的对不对好，但我们如果做这个课程安排，我们当然还可以安排一个运用这个规律的，把这个规律运用起来。所以整一个环节你会发现就是一个发现探索的一个教学的过程的。赫布鲁纳提出的教学模式来可以支持我这一个教学过程的设计，对不对啊好，那么对于原理的教学，我们知道有这两种模式，当然还有其他很多的，你们一定能够找到的啊，那今天时间关系我们就呈现这两种。那我也知道我可以用在我的教学过程设计当中去，其实你还是要知道一些原理的知识。其实成人和大学生，包括你们在内，你们在学习一些原理的时候，往往都是要用有意义的接受式的学习。为什么？效率高。对有效性高的，但是对于儿童来说，10到12岁就是四年级的时候啊。有的孩子可以提前，如果教育方法得到，可以提前到三年级是他的演绎和归纳那种思维的一个高速发展期啊。一个加速期的。所以在这一个时期，我们可以提供这样的探索发现的过程，来培养他们的归纳的思维和演绎推理的思维啊，培养他的数学思维能力的。所以我们可以创设这样的啊一种教学的流程的好，那么这样我们就通过2个课例。4种模式带大家感受了一下教学模式的一个选择和应用的过程，我们需要考虑到它的一些影响因素的，然后有针对性的去选择指导我们的过程的设计的好，那同学们可能会问了，我会联想到说刘老师你和杜老师始终给我们强调要在信息化的环境之下考虑我们的教学设计啊，因为你们师范专业认证就3级认证就提出要培养你们的信息化教学能力，还有国家的许多其他文件都有这种要求，所以我们倒逼你们去思考这个问题。在智慧教室的环境之下啊那么国家在2014年颁布了中小学教师信息技术应用能力的标准他给老师呢提出了两条两条建议路径就是来做这种计时技术呃融合教学的一种呢就是用老师的这个技术呢去优化课堂教学另外一个就是转变老师的呃转变学生的学习方式那如果是按照第一种路径来做我们的教学模式完全可以不用变它仍然是原来的模式只是说在某一些环节我融入进组来优化。我这个环节，比如说刚才同学们提到这个投屏就可以优化，比如说这地方我可以用动画更加直观，这就是优化的，我不改变原来模式的结构的，但如果我想改变学生的学习方式，就彻底的改变了，那这时候你的教学模式的结构会发生转变好。那比如说我们这一节课用的是翻转的意味，有课前的资源推送，然后测试课中讲解我们下节课还会用技术来继续支持远程教学，然后最后客户推送资源的。那同学可能问，那我的课可不可以用？翻转课堂当然可以用，只要他能够提高你的效率是合理的。你可以用包括刚才的认识平行四边形那个课时的内容，其实是很满的。除了讲定义之外，还要会去画高，那你可以去考虑我可不可以有一部分内容前置到课前学，那么刻钟的时候。老，你可能问，老师，我可不可以仍然用童话和形成的路径来交，当然是可以的，只要合理，我的模式是可以累加的，可以迭代的好。那这就是呃，技术支持之下的教学模式。那这地方呢，就推荐谢老师和尹老师的。呃，这个网络化教学设计，同学们可以结合尹老师那个慕课，再去感受一下智慧课堂之下的教学模式的样态，发生了其他的一种转变的，可以继续学习。那么我们通过前面的这样的很长时间的一个深化学习。我们可以回答我们最初的这一个问题了。如何来设计小学数学教学模式？我们来回忆一下，首先你一定要有知识储备，这就是大学赋予你们的精神力量，你们一定要有这种知识储备。在你们的教学实践过程当中，你才能够随时的调取出来啊。你们之前学的课程，包括今天介绍的你的模式的储备是否已经足够了？那么接下来你用在你的教学设计当中，就会多一条选择的路径，不再是纯粹基于经验和基于他人的教学设计了啊。然后我分析教材。学琴目标，这都是我的影响因素。考虑清楚之后，我去选择合理的恰当的教学模式，选择之后呢？我来看，因为一个模式，它包括纵向的环节，那我可以进行微调，符合我的教学的次序，我要调整这个教学程序，然后教学模式，他对每一个环节他是有解毒的，这个环节老师应该做什么事情，他也有教学策略和评价的，那我就可以用它来安排我的教学活动来设计教学活动，融入一些方法手段，这样最后就形成了。纵横交错的一个稳定的结构形式好，那么这就是一个教学模式用在我的过程设计当中，那如果同学们有兴趣，可以看这一本书，就里病人老师的教学论研究20年，这本书很好，里面有专门的板块，是讲教学模式，如何选择，如何应用，如何我们将来作为一名创造者来构建一个教学模式，它都有详细的理论介绍的啊，同学们有兴趣可以拓展学习，当然还有其他的一些文献的好，那我们这一节课的理论学习部分就到此结束。接下来我们就要学以致用，在教学的实践环节当中，去体验一下教学模式的选择和使用。好。同学们好。我们回到课堂上来，那我们这一节课呢，就是学以致用，把前面学到的技术方法。'

In [ ]:
pattern = re.compile(u'\t|\n|\.|-|:|,|;|。|\)|\(|\?|"') 

In [ ]:
string_data = re.sub(pattern, '', strContent)

In [ ]:
string_data